In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'

embed_size=256
dropout=0
block_size=1024

In [5]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.query=nn.Linear(embed_size, head_size, bias=False)
    self.key=nn.Linear(embed_size, head_size, bias=False)
    self.value=nn.Linear(embed_size, head_size, bias=False)
    self.dropout=nn.Dropout(dropout)

    #caching:
    self.k_cache=None
    self.v_cache=None
    self.cache_index=0

  def forward(self, x):
    B, T, C=x.shape
    q=self.query(x)
    k=self.key(x)
    v=self.value(x)

    #initializing KV Caching if empty:
    if self.k_cache is None or self.v_cache is None:
      #block size is the context length(for prediction of next token, the max no of tokens, current token can look back up to ) and it will be the max no of tokens we will be caching
      self.k_cache=torch.zeros(B, block_size, self.head_size, device=x.device)
      self.v_cache=torch.zeros(B, block_size, self.head_size, device=x.device)
      self.cache_index=0

    #Updating the cache:
    if self.cache_index + T <=block_size:
      self.k_cache[:, self.cache_index:self.cache_index+T, :]=k
      self.v_cache[:, self.cache_index:self.cache_index+T, :]=v

      #Note: here we pass only one token at a time, T will always be 1
      #the code above are equivalent to performing self.k_cache[:, self.cache_index, :]=k
    else:
      #shifting one token back:
      shift= self.cache_index + T - block_size #shift is always 1
      self.k_cache[:, :-shift, :]=self.k_cache[:, shift:, :].clone()
      self.v_cache[:, :-shift, :]=self.v_cache[:, shift:, :].clone()

      #store new values:
      self.k_cache[:, -T:, :]=k
      self.v_cache[:, -T, :]=v

    #Updating Cache index:
    self.cache_index=min(self.cache_index + T, block_size)

    wei= q @ self.k_cache.transpose(2, 1) / self.head_size **0.5
    #masking is necessary if using multiple tokens decoding at once
    wei=wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
    wei=F.softmax(wei, dim=2)
    wei=self.dropout(wei)
    out=wei @ self.v_cache

    return out

In [ ]:
k_cache = torch.zeros(1, 3, 3)
v_cache = torch.zeros(1, 3, 3)

steps = 3
for i in range(steps):
  k_cache[:, i, :] = torch.randint(10, (1, 3))
print("k_cache Before:\n", k_cache)

shift = 1
k_cache[:, :-shift, :] = k_cache[:, shift:, :].clone()
v_cache[:, :-shift, :] = v_cache[:, shift:, :].clone()
print("k_cache After:\n", k_cache)

k_cache Before:
 tensor([[[2., 2., 5.],
         [1., 0., 3.],
         [8., 5., 2.]]])
k_cache After:
 tensor([[[1., 0., 3.],
         [8., 5., 2.],
         [8., 5., 2.]]])


## Inferencing:

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import time

In [11]:
#@ Model loading and tokenizer:
model_name='gpt2'
model=AutoModelForCausalLM.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

#@ Text Prompt:
prompt="Once upon a time"
input_ids=tokenizer(prompt, return_tensors="pt").input_ids

#@ Function to generate text and measure time
def generate_text(use_cache):
  start_time=time.time()
  output_ids = model.generate(input_ids, use_cache=use_cache, max_new_tokens=1000).to(device)
  elapsed_time=time.time()-start_time

  #decoding the output:
  output_text=tokenizer.decode(output_ids[0], skip_special_tokens=True)

  return output_text, elapsed_time

# Generate without KV cache
output_without_cache, time_without_cache = generate_text(use_cache=False)
print("Without KV Cache:")
print(output_without_cache)
print(f"Time taken: {time_without_cache:.2f} seconds\n")

# Generate with KV cache
output_with_cache, time_with_cache = generate_text(use_cache=True)
print("With KV Cache:")
print(output_with_cache)
print(f"Time taken: {time_with_cache:.2f} seconds")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KeyboardInterrupt: 